In [1]:
# %load main.py
import tensorflow as tf
import numpy as np
import os, argparse, time, random
from utils import str2bool, get_logger, get_entity
from data import read_corpus, read_dictionary, tag2label, random_embedding

from model import BiLSTM_CRF

C:\Users\Ian\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
## Session configuration
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # default: 0
config = tf.ConfigProto()
# config.gpu_options.allow_growth = True
# config.gpu_options.per_process_gpu_memory_fraction = 0.2  # need ~700MB GPU memory

In [3]:
## hyperparameters
parser = argparse.ArgumentParser(description='BiLSTM-CRF for Chinese NER task')
parser.add_argument('--train_data', type=str, default='data_path', help='train data source')
parser.add_argument('--test_data', type=str, default='data_path', help='test data source')
parser.add_argument('--batch_size', type=int, default=64, help='#sample of each minibatch')
parser.add_argument('--epoch', type=int, default=40, help='#epoch of training')
parser.add_argument('--hidden_dim', type=int, default=300, help='#dim of hidden state')
parser.add_argument('--optimizer', type=str, default='Adam', help='Adam/Adadelta/Adagrad/RMSProp/Momentum/SGD')
parser.add_argument('--CRF', type=str2bool, default=True, help='use CRF at the top layer. if False, use Softmax')
parser.add_argument('--lr', type=float, default=0.001, help='learning rate')
parser.add_argument('--clip', type=float, default=5.0, help='gradient clipping')
parser.add_argument('--dropout', type=float, default=0.5, help='dropout keep_prob')
parser.add_argument('--update_embedding', type=str2bool, default=True, help='update embedding during training')
parser.add_argument('--pretrain_embedding', type=str, default='random', help='use pretrained char embedding or init it randomly')
parser.add_argument('--embedding_dim', type=int, default=300, help='random init char embedding_dim')
parser.add_argument('--shuffle', type=str2bool, default=True, help='shuffle training data before each epoch')
parser.add_argument('--mode', type=str, default='demo', help='train/test/demo')
parser.add_argument('--demo_model', type=str, default='1521112368', help='model for test and demo')
args = parser.parse_args()

usage: ipykernel_launcher.py [-h] [--train_data TRAIN_DATA]
                             [--test_data TEST_DATA] [--batch_size BATCH_SIZE]
                             [--epoch EPOCH] [--hidden_dim HIDDEN_DIM]
                             [--optimizer OPTIMIZER] [--CRF CRF] [--lr LR]
                             [--clip CLIP] [--dropout DROPOUT]
                             [--update_embedding UPDATE_EMBEDDING]
                             [--pretrain_embedding PRETRAIN_EMBEDDING]
                             [--embedding_dim EMBEDDING_DIM]
                             [--shuffle SHUFFLE] [--mode MODE]
                             [--demo_model DEMO_MODEL]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\Ian\AppData\Roaming\jupyter\runtime\kernel-1031f66c-1f35-4276-8431-9d962fca18db.json


SystemExit: 2

C:\Users\Ian\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
## get char embeddings
word2id = read_dictionary(os.path.join('.', 'data_path', 'word2id.pkl'))
type(word2id)

vocab_size: 3905


dict

In [6]:
word2id.keys()

dict_keys(['当', '希', '望', '工', '程', '救', '助', '的', '百', '万', '儿', '童', '成', '长', '起', '来', '，', '科', '教', '兴', '国', '蔚', '然', '风', '时', '今', '天', '有', '收', '藏', '价', '值', '书', '你', '没', '买', '明', '日', '就', '叫', '悔', '不', '初', '！', '本', '是', '所', '传', '统', '门', '类', '中', '第', '一', '大', '户', '只', '我', '们', '结', '束', '温', '饱', '间', '太', '短', '而', '已', '。', '因', '关', '寇', '在', '京', '掠', '夺', '文', '物', '详', '情', '界', '较', '为', '重', '视', '也', '北', '史', '料', '要', '件', '之', '册', '<NUM>', '年', '月', '油', '印', '《', '保', '存', '管', '状', '态', '调', '查', '报', '告', '》', '范', '围', '涉', '及', '故', '宫', '、', '历', '博', '古', '研', '清', '华', '图', '馆', '伪', '资', '库', '等', '二', '十', '几', '家', '言', '以', '上', '洋', '三', '余', '珍', '贵', '乡', '献', '特', '定', '期', '刊', '某', '名', '或', '著', '多', '种', '出', '版', '专', '题', '注', '意', '精', '品', '非', '卖', '纪', '念', '集', '系', '列', '那', '过', '经', '够', '您', '玩', '味', '无', '穷', '了', '受', '到', '郑', '振', '铎', '先', '生', '阿', '英', '作', '启', '示', '从', '个', '人', '条', '发', '瞄', '准', '现', 

In [5]:
# if args.pretrain_embedding == 'random':
#     embeddings = random_embedding(word2id, args.embedding_dim)
# else:
#     embedding_path = 'pretrain_embedding.npy'
#     embeddings = np.array(np.load(embedding_path), dtype='float32')
embeddings = random_embedding(word2id, 300)
type(embeddings),embeddings.shape

(numpy.ndarray, (3905, 300))

In [9]:
embeddings[:1,:]

array([[ 0.24289457,  0.08744171, -0.19475493, -0.17992818, -0.05061282,
        -0.05096334, -0.10729764,  0.19720572, -0.00817338, -0.02671687,
        -0.21701367, -0.14643739,  0.10569254, -0.24897547, -0.24331471,
         0.05571439,  0.068689  ,  0.03074452,  0.00349323, -0.19363198,
         0.19154462,  0.08363545, -0.00707453, -0.03918197,  0.08813415,
         0.06555788, -0.09478968,  0.04997296,  0.0188967 ,  0.18829729,
        -0.17980649,  0.22334029, -0.05274172,  0.10673455, -0.06408237,
         0.22457956,  0.03680328, -0.1484314 , -0.03141019,  0.05154869,
        -0.12299992, -0.22925493, -0.05336275,  0.01276062, -0.18880174,
         0.1003283 ,  0.0598359 , -0.08483516,  0.08332489,  0.1453224 ,
         0.01927098, -0.20168571,  0.24328902,  0.21241413,  0.18247628,
         0.07354892,  0.06478015,  0.06478026,  0.05219268, -0.02274675,
         0.02637642,  0.07520191, -0.00105785, -0.02584241,  0.14295278,
         0.17856646, -0.16664007,  0.14560308, -0.2

In [6]:
## read corpus and get training data
# if args.mode != 'demo':
#     train_path = os.path.join('.', args.train_data, 'train_data')
#     test_path = os.path.join('.', args.test_data, 'test_data')
#     train_data = read_corpus(train_path)
#     test_data = read_corpus(test_path)
#     test_size = len(test_data)
train_path = os.path.join('.', 'data_path', 'train_data')
test_path = os.path.join('.', 'data_path', 'test_data')
train_data = read_corpus(train_path)
test_data = read_corpus(test_path)
test_size = len(test_data)
len(test_data),test_data[:2]

(4631,
 [(['中',
    '共',
    '中',
    '央',
    '致',
    '中',
    '国',
    '致',
    '公',
    '党',
    '十',
    '一',
    '大',
    '的',
    '贺',
    '词'],
   ['B-ORG',
    'I-ORG',
    'I-ORG',
    'I-ORG',
    'O',
    'B-ORG',
    'I-ORG',
    'I-ORG',
    'I-ORG',
    'I-ORG',
    'I-ORG',
    'I-ORG',
    'I-ORG',
    'O',
    'O',
    'O']),
  (['各', '位', '代', '表', '、', '各', '位', '同', '志', '：'],
   ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'])])

In [7]:
## paths setting
paths = {}
# timestamp = str(int(time.time())) if args.mode == 'train' else args.demo_model
timestamp = str(int(time.time()))
output_path = os.path.join('.', 'data_path'+"_save", timestamp)
if not os.path.exists(output_path): os.makedirs(output_path)
summary_path = os.path.join(output_path, "summaries")
paths['summary_path'] = summary_path
if not os.path.exists(summary_path): os.makedirs(summary_path)
model_path = os.path.join(output_path, "checkpoints/")
if not os.path.exists(model_path): os.makedirs(model_path)
ckpt_prefix = os.path.join(model_path, "model")
paths['model_path'] = ckpt_prefix
result_path = os.path.join(output_path, "results")
paths['result_path'] = result_path
if not os.path.exists(result_path): os.makedirs(result_path)
log_path = os.path.join(result_path, "log.txt")
paths['log_path'] = log_path
# get_logger(log_path).info(str(args))
get_logger(log_path).info('BiLSTM-CRF for Chinese NER task')

BiLSTM-CRF for Chinese NER task


In [8]:
## training model
# if args.mode == 'train':
#     model = BiLSTM_CRF(args, embeddings, tag2label, word2id, paths, config=config)
#     model.build_graph()

#     ## hyperparameters-tuning, split train/dev
#     # dev_data = train_data[:5000]; dev_size = len(dev_data)
#     # train_data = train_data[5000:]; train_size = len(train_data)
#     # print("train data: {0}\ndev data: {1}".format(train_size, dev_size))
#     # model.train(train=train_data, dev=dev_data)

#     ## train model on the whole training data
#     print("train data: {}".format(len(train_data)))
#     model.train(train=train_data, dev=test_data)  # use test_data as the dev_data to see overfitting phenomena


# model = BiLSTM_CRF(args, embeddings, tag2label, word2id, paths, config=config)
model = BiLSTM_CRF(embeddings, tag2label, word2id, paths=paths, config=config)
model.build_graph()

## hyperparameters-tuning, split train/dev
# dev_data = train_data[:5000]; dev_size = len(dev_data)
# train_data = train_data[5000:]; train_size = len(train_data)
# print("train data: {0}\ndev data: {1}".format(train_size, dev_size))
# model.train(train=train_data, dev=dev_data)

## train model on the whole training data
print("train data: {}".format(len(train_data)))
model.train(train=train_data, dev=test_data)  # use test_data as the dev_data to see overfitting phenomena
# tf.nn.embedding_lookup()

C:\Users\Ian\Anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


train data: 50658


2018-05-21 16:10:10 epoch 1, step 1, loss: 81.3, global_step: 1


2018-05-21 16:10:10 epoch 1, step 300, loss: 6.431, global_step: 300


2018-05-21 16:10:10 epoch 1, step 600, loss: 4.385, global_step: 600


...

2018-05-22 08:28:22 epoch 39, step 792, loss: 0.08669, global_step: 30888
===========validation / test===========


2018-05-22 08:54:37 epoch 40, step 1, loss: 0.09028, global_step: 30889


2018-05-22 08:54:37 epoch 40, step 300, loss: 0.06054, global_step: 31188


2018-05-22 08:54:37 epoch 40, step 600, loss: 0.04246, global_step: 31488


2018-05-22 08:54:37 epoch 40, step 792, loss: 0.02431, global_step: 31680
===========validation / test===========


In [12]:
## testing model
# elif args.mode == 'test':
#     ckpt_file = tf.train.latest_checkpoint(model_path)
#     print(ckpt_file)
#     paths['model_path'] = ckpt_file
#     model = BiLSTM_CRF(args, embeddings, tag2label, word2id, paths, config=config)
#     model.build_graph()
#     print("test data: {}".format(test_size))
#     model.test(test_data)
## testing model
ckpt_file = tf.train.latest_checkpoint(model_path)
print(ckpt_file)
paths['model_path'] = ckpt_file
# model = BiLSTM_CRF(embeddings, tag2label, word2id, paths, config=config)
# model.build_graph()
print("test data: {}".format(test_size))
model.test(test_data)

=========== testing ===========


.\data_path_save\1526890202\checkpoints/model-31680
test data: 4631
INFO:tensorflow:Restoring parameters from .\data_path_save\1526890202\checkpoints/model-31680


Restoring parameters from .\data_path_save\1526890202\checkpoints/model-31680


NotFoundError: Key words_2/_word_embeddings not found in checkpoint
	 [[Node: save_1/RestoreV2_28 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save_1/Const_0_0, save_1/RestoreV2_28/tensor_names, save_1/RestoreV2_28/shape_and_slices)]]

Caused by op 'save_1/RestoreV2_28', defined at:
  File "C:\Users\Ian\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Ian\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Ian\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Ian\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\Ian\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 478, in start
    self.io_loop.start()
  File "C:\Users\Ian\Anaconda3\lib\site-packages\zmq\eventloop\ioloop.py", line 177, in start
    

In [10]:
## demo
# elif args.mode == 'demo':
#     ckpt_file = tf.train.latest_checkpoint(model_path)
#     print(ckpt_file)
#     paths['model_path'] = ckpt_file
#     model = BiLSTM_CRF(args, embeddings, tag2label, word2id, paths, config=config)
#     model.build_graph()
#     saver = tf.train.Saver()
#     with tf.Session(config=config) as sess:
#         print('============= demo =============')
#         saver.restore(sess, ckpt_file)
#         while(1):
#             print('Please input your sentence:')
#             demo_sent = input()
#             if demo_sent == '' or demo_sent.isspace():
#                 print('See you next time!')
#                 break
#             else:
#                 demo_sent = list(demo_sent.strip())
#                 demo_data = [(demo_sent, ['O'] * len(demo_sent))]
#                 tag = model.demo_one(sess, demo_data)
#                 PER, LOC, ORG = get_entity(tag, demo_sent)
#                 print('PER: {}\nLOC: {}\nORG: {}'.format(PER, LOC, ORG))
ckpt_file = tf.train.latest_checkpoint(model_path)
print(ckpt_file)
paths['model_path'] = ckpt_file
model = BiLSTM_CRF(embeddings, tag2label, word2id, paths, config=config)
model.build_graph()
saver = tf.train.Saver()
with tf.Session(config=config) as sess:
    print('============= demo =============')
    saver.restore(sess, ckpt_file)
    while(1):
        print('Please input your sentence:')
        demo_sent = input()
        if demo_sent == '' or demo_sent.isspace():
            print('See you next time!')
            break
        else:
            demo_sent = list(demo_sent.strip())
            demo_data = [(demo_sent, ['O'] * len(demo_sent))]
            tag = model.demo_one(sess, demo_data)
            PER, LOC, ORG = get_entity(tag, demo_sent)
            print('PER: {}\nLOC: {}\nORG: {}'.format(PER, LOC, ORG))



.\data_path_save\1526890202\checkpoints/model-31680


ValueError: Variable bi-lstm/bidirectional_rnn/fw/lstm_cell/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "C:\Users\Ian\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 1470, in __init__
    self._traceback = self._graph._extract_stack()  # pylint: disable=protected-access
  File "C:\Users\Ian\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 2956, in create_op
    op_def=op_def)
  File "C:\Users\Ian\Anaconda3\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 787, in _apply_op_helper
    op_def=op_def)


In [11]:
np.save('pretrain_embedding.npy',model.embeddings)